In [1]:
import graphlab
import random
import math
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to rosaljoao@gmail.com and will expire on March 22, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1525355680.log


In [4]:
random.seed(98103)
n = 30
x = graphlab.SArray([random.random() for i in range(n)]).sort()

In [5]:
y = x.apply(lambda x: math.sin(4*x))

In [6]:
random.seed(1)
e = graphlab.SArray([random.gauss(0,1.0/3.0) for i in range(n)])
y = y + e

In [7]:
data = graphlab.SFrame({'X1':x, 'Y':y})

In [8]:
def polyomial_features(data, deg):
    data_copy.copy()
    for i in range(1,deg):
        data_copy['X'+str(i+1)]=data_copy['X'+str(i)]*data_copy['X1']
    return data_copy

In [14]:
def polynomial_lasso_regression(data, deg, l1_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg),
        target='Y', l2_penalty=0.,
        l1_penalty=l1_penalty,
        validation_set=None,
        solver='fista', verbose=False,
        max_iterations=3000,
        convergence_threshold=1e-10)
    return model

In [15]:
def print_coefficients(model):
    deg = len(model.coefficients['value'])-1
    w = list(model.coefficients['value'])
    
    print 'Coeficientes do polinômio de grau' + srt(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [20]:
def plot_data(data):
    plt.plot(data['X1'],data['Y'],'k.')
    plt.xlabel('x')
    plt.ylabel('y')
    
def plot_poly_predictions(data, model):
    plot_data(data)
    deg = len(model.coefficients['value'])-1
    x_pred = graphlab.SFrame({'X1':[i/200.0 for i in range(200)]})
    y_pred = model.predict(polynomial_features(x_pred,deg))
    plt.plot(x_pred['X1'], y_pred, 'g-', label='degree ' + str(deg) + ' fit')
    plt.legend(loc='upper left')
    plt.axis([0,1,-1.5,2])
    for l1_penalty in [0.0001, 0.01, 0.1, 10]:
        model = polynomial_lasso_regression(data, deg=16, l1_penalty=l1_penalty)
        print 'l1_penalty = %e' % l1_penalty
        print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
        print_coefficients(model)
        print '\n'
        plt.figure()
        plot_poly_predictions(data,model)
        plt.title('LASSO, lambda = %.2e, # nonzeros = %d' % (l1_penalty,
        (model.coefficients['value']).nnz()))    
